# Get Standup Show Ratings

# IMBD API

## Imports

In [18]:
import requests
import numpy as np
import pandas as pd
import re
import json
from datetime import date

import pickle

import plotly.express as px

In [3]:
tscripts = pd.read_pickle('../data/transcripts_raw_df.pickle')
tscripts['script characters'] = tscripts['transcript'].apply(len)
px.strip(tscripts, x = 'script characters', hover_data= ['description'])


In [4]:
tscripts = tscripts[tscripts['script characters'] > 21000]
shows_split = tscripts['description'].str.split('|', expand = True)[0]

In [5]:
def transcript_strip(string):
    hasyear = re.search(r"\(\d{4}\)", string)
    hasdashbracket = re.search(r"\s[–\[]", string)
    if hasyear:
        return string[:hasyear.end()]
    elif hasdashbracket:
        return string[:hasdashbracket.start()]
    else:
        return string

In [6]:
show_searches =  [transcript_strip(search) for search in shows_split]

## Get IDs from API
I'll have to only do 100 results per day since I'm sticking with the free version of the API.
After I get all the title IDs (4 days), I'll then have to get the ratings (another 4 days). If I want the reviews, that's another 4 days.

In [203]:
api_key = json.load(open('../../../api_keys/api_keys.json'))['imdb2']

In [87]:
# with open(f'../data/imdb_title_search_results_chris_2022-05-23.pickle', 'rb') as file:
#     search_results = pickle.load(file)

# with open(f'../data/imdb_title_search_results_fails_chris_2022-05-23.pickle', 'rb') as file:
#     search_results_fails = pickle.load(file)

In [31]:
start = None
end = None
for search in show_searches[start:end]:

    try:
        endpoint = 'https://imdb-api.com/en/API/SearchTitle/'
        params = {
            'apiKey':api_key,
            'expression':search
        }
        response = requests.get(endpoint, params)
        result = response.json()

        search_result = result['results'][0]
        search_results.append(search_result)
    except:
        search_results_fails.append(search)

In [41]:
# with open(f'../data/imdb_title_search_results_{date.today()}.pickle', 'wb') as file:
#     pickle.dump(search_results, file)

# with open(f'../data/imdb_title_search_results_fails_{date.today()}.pickle', 'wb') as file:
#     pickle.dump(search_results_fails, file)

## Get Title Info from API

In [45]:
# title_results = []

In [123]:
# title_results_fails = []
# with open(f'../data/imdb_title_results_{date.today()}.pickle', 'rb') as file:
#     title_results = pickle.load(file)

start = 299
end = None
for search_dict in search_results[start:end]:

    try:
        endpoint = 'https://imdb-api.com/en/API/Title/'
        params = {
            'apiKey': api_key,
            'id': search_dict['id']
        }
        response = requests.get(endpoint, params)
        title_result = response.json()

        title_results.append(title_result)
    except:
        title_results_fails.append(search_dict['title'])

In [156]:
# with open(f'../data/imdb_title_results_{date.today()}.pickle', 'wb') as file:
#     pickle.dump(title_results, file)

## Get Reviews from API
Unfortunately, these appear capped at 25.

In [160]:
endpoint = 'https://imdb-api.com/en/API/Reviews/'
params = {
    'apiKey': api_key,
    'id': search_results[0]['id']
}
response = requests.get(endpoint, params)
result = response.json()
result

{'imDbId': 'tt15907298',
 'title': 'Jim Gaffigan: Comedy Monster',
 'fullTitle': 'Jim Gaffigan: Comedy Monster (2021)',
 'type': 'Movie',
 'year': '2021',
 'items': [{'username': 'Johnny_West',
   'userUrl': 'https://www.imdb.com/user/ur57178464',
   'reviewLink': 'https://www.imdb.com/review/rw7694078',
   'warningSpoilers': False,
   'date': '27 December 2021',
   'rate': '',
   'helpful': '10 out of 13 found this helpful.',
   'title': 'Not his best',
   'content': 'I have watched every Jim Gaffigan special on Netflix. This special had its funny moments, but there were various spots where Gaffigan was not connecting, and the jokes were just observations. Not funny or unfunny, just nothing.The pandemic jokes were stale. Gaffigan also hit being overweight and his food obsessions. He does that in every comedy special. He usually has some new spin on those topics, to make it fresh and funny. This time, it sounded really tired.I can only say that there are some Gaffigan specials that I h

In [186]:
# review_results_fails = []
# review_results = []

start = 279
end = None
for search_dict in search_results[start:end]:

    try:
        endpoint = 'https://imdb-api.com/en/API/Reviews/'
        params = {
            'apiKey': api_key,
            'id': search_dict['id']
        }
        response = requests.get(endpoint, params)
        review_result = response.json()

        review_results.append(review_result)
    except:
        review_results_fails.append(search_dict['title'])

In [187]:
len(review_results)

314

In [188]:
with open(f"../data/imdb_review_results_{date.today()}", 'wb') as file:
    pickle.dump(review_results, file)

In [189]:
# max number of reviews is 25. this must be capped.
max([len(rev_res['items']) for rev_res in review_results])

TypeError: object of type 'NoneType' has no len()

## Try those short titles

In [195]:
tscripts = pd.read_pickle('../data/transcripts_raw_df.pickle')
tscripts['script characters'] = tscripts['transcript'].apply(len)
short_tscripts = tscripts[tscripts['script characters'] < 21000]

short_shows_split = short_tscripts['description'].str.split('|', expand = True)[0]
short_show_searches =  [transcript_strip(search) for search in short_shows_split]

In [197]:
# short_search_results = []
# short_search_results_fails = []

In [204]:
start = None
end = 57
for search in short_show_searches[start:end]:

    try:
        endpoint = 'https://imdb-api.com/en/API/SearchTitle/'
        params = {
            'apiKey':api_key,
            'expression':search
        }
        response = requests.get(endpoint, params)
        result = response.json()

        short_search_result = result['results'][0]
        short_search_results.append(short_search_result)
    except:
        short_search_results_fails.append(search)

In [212]:
short_title_results = []
short_title_results_fails = []

for search_dict in short_search_results:

    try:
        endpoint = 'https://imdb-api.com/en/API/Title/'
        params = {
            'apiKey': api_key,
            'id': search_dict['id']
        }
        response = requests.get(endpoint, params)
        short_title_result = response.json()

        short_title_results.append(short_title_result)
    except:
        short_title_results_fails.append(search_dict['title'])

## Reconciliation

In [221]:
empty_search_dict = dict(zip(title_results[0].keys(), [None for i in range(len(title_results[0].keys()))]))

In [ ]:
for entry in short_search_results_fails:
    ind = short_show_searches.index(entry)
    short_title_results.insert(ind, empty_search_dict)

In [235]:
full_title_results = title_results + short_title_results
for entry in search_results_fails:
    ind = show_searches.index(entry)
    full_title_results.insert(ind, empty_search_dict)

In [234]:
len(short_title_results)

57

In [236]:
len(full_title_results)

381

In [237]:
with open(f"../data/imdb_full_title_results_{date.today()}.pickle", 'wb') as file:
    pickle.dump(full_title_results, file)

In [241]:
tscripts_df_cut = pd.concat([tscripts[tscripts['script characters'] > 21000], tscripts[tscripts['script characters'] < 21000]]).reset_index(drop = True)
tscripts_df_cut

,description,link,transcript,script characters
0,Jim Gaffigan: Comedy Monster (2021) | Transcript,https://scrapsfromtheloft.com/comedy/jim-gaffi...,"Thank you! Thank you! Oh, my gosh. Thank you s...",49799
1,Louis C. K.: Sorry (2021) | Transcript,https://scrapsfromtheloft.com/comedy/louis-c-k...,♪♪ [“Like a Rolling Stone” by Bob Dylan playin...,44669
2,Drew Michael: Drew Michael (2018) | Transcript,https://scrapsfromtheloft.com/comedy/drew-mich...,“This is the latest I’ve stayed up in a long t...,40006
3,Drew Michael: Red Blue Green (2021) | Transcript,https://scrapsfromtheloft.com/comedy/drew-mich...,(EMOTIONAL MUSIC PLAYING) (MUSIC ENDS) DREW MI...,50422
4,Mo Amer: Mohammed in Texas (2021) | Transcript,https://scrapsfromtheloft.com/comedy/mo-amer-m...,[quirky flute music playing] [single note pian...,58020
...,...,...,...,...
376,NORM MACDONALD: ONE NIGHT STAND (1991) – Full ...,https://scrapsfromtheloft.com/comedy/norm-macd...,"I BIT MY TONGUE Well, good to see ya- good cro...",19179
377,Hasan Minhaj at 2017 White House Correspondent...,https://scrapsfromtheloft.com/comedy/hasan-min...,Hasan Minhaj’s full comedy routine at the 2017...,18433
378,George Carlin: Seven Words You Can Never Say o...,https://scrapsfromtheloft.com/comedy/george-ca...,The original version of George Carlin’s classi...,5630
379,Seth Meyers at the 2011 White House Correspond...,https://scrapsfromtheloft.com/comedy/seth-meye...,"In April 2011, Saturday Night Live’s Seth Meye...",16156


In [242]:
with open(f"../data/transcripts_df_{date.today()}.pickle", 'wb') as file:
    pickle.dump(tscripts_df_cut, file)